In [6]:
############********** Project1 Task1 Training File **********############


import pyodbc
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


# pearson's correlation feature selection for numeric input and numeric output
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.datasets import make_classification
from sklearn_pandas import CategoricalImputer ###########
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from matplotlib import pyplot
import datetime as dt

from datetime import date
import holidays

pd.set_option("display.max_columns", None)


In [7]:
###--- Connecting to the server to access the database ---###

conn = pyodbc.connect('Driver={SQL Server};'
'Server=HCV171DTMBPDA01\DTMBSQL16PRD.NGDS.state.mi.us,1910;'
'Database=DTMB_CentralProcurement;'
'Trusted_Connection=yes;')

cursor = conn.cursor()

In [8]:
###--- Retrieving tables from the database using queries ---###

query_1 = "SELECT * FROM tblProjects WHERE ProjectType = 'RFP' Or ProjectType = 'C-RFP' Or ProjectType = 'CN' Or ProjectType = 'DS' Or ProjectType = 'CO-OP'Or ProjectType = 'ITN'Or ProjectType = 'CPC' Or ProjectType = 'RFS '"
query_2 = "SELECT * FROM tblContractCategories"


df_projects = pd.read_sql_query(query_1, conn)
df_projects = df_projects[df_projects['CategoryID'].notna()]
df_projects = df_projects.astype({'CategoryID': 'int64'})

df_level = pd.read_sql_query(query_2, conn)
df_level = df_level[['LevelCalc', 'CategoryID']]

df = df_projects.join(df_level.set_index('CategoryID'), on='CategoryID')
df = df[(df["ProjectStage"] == 'Complete') ]
df.head()


,ID,SharePointID,ProjectColor,ProjectName,ProjectType,CreateDate,CreatedBy,PlanStartDate,PlanFinishDate,StatusDate,ProjectDescription,DollarAmount,Status,WorkPerformedFor,RefrenceNumber,ProjectStage,CancelationReason,BuyerName,Hours,SensitivityIndicator,Division,SourcingCategory,ActualFinishDate,Unit,SLA,Planned,SolicitationStatus,LastUpdatedBy,LastUpdateDate,LastUpdatedField,CategoryID,FedReview,ITComponent,SolicitationNum,LevelCalc
2,3339,3339,Green,"Segal Advisors, Inc",CN,2017-07-17,keilenn,2017-07-17,2017-10-12,2017-07-17,The State of Michigan is requesting an amendme...,48800.00,Received Chapter 5 Approval on 07/17/2017. In...,071,C20170714-131440,Complete,0.0,JY1,0.0,False,PROF,"Consulting, Financial",2017-07-24,NaN,63,Yes,None,testM,2019-03-06 16:23:21,", Project Stage",50,False,False,None,3
3,3570,3570,Green,2018 Heavy Truck Annual Pricing,RFP,2017-07-17,Gibrilm,2017-08-16,2017-11-17,2017-11-06,This request is to obtain 2018 Heavy Truck Ann...,0.00,Complete,000,None,Complete,0.0,YMD,120.0,False,COMM,"Motor Vehicles, Heavy Trucks",2017-11-02,NaN,138,Yes,New-Bid,testM,2019-03-06 16:25:28,", Project Stage, Dollar Amount",185,False,False,None,2
4,3571,3571,Green,2018 Vehicle Model Annual Pricing,RFP,2017-07-17,Gibrilm,2017-08-14,2017-11-17,2017-11-06,This request is for 2018 Vehicle Model Annual ...,NaN,Complete.,000,None,Complete,0.0,YMD,120.0,False,COMM,"Motor Vehicles, General",2017-11-06,NaN,138,None,New-Bid,None,NaT,None,184,False,False,None,3
6,3574,3574,Green,Aerial Towers and Accessories,RFP,2017-07-17,Gibrilm,2017-03-02,2017-09-17,2017-06-12,Purchase and rentals of Aerial Towers and acce...,5000000.00,Multi award. Both contracts executed.,591,C20170209-090304,Complete,0.0,YMD,120.0,False,COMM,"Motor Vehicle, Accessories",2017-07-25,NaN,138,None,None,None,NaT,None,301,False,False,None,2
11,3580,3580,Green,"Cholestech, Afinion and iFOBT Equip. and Test...",RFP,2017-07-17,Gibrilm,2017-05-18,2017-12-03,2018-01-29,DHHS Cancer Prevention Section requests to est...,182413.44,New contract has been executed. Request compl...,431,C20170512-141419,Complete,0.0,SR4,120.0,False,COMM,Lab Equipment & Supplies,2018-02-01,NaN,138,None,Re-Bid,None,NaT,None,143,False,False,None,4


In [9]:
X, y = make_classification(n_samples=100, n_features=20, n_informative=2)
# define feature selection
fs = SelectKBest(score_func=f_classif, k=2)
# apply feature selection
X_selected = fs.fit_transform(X, y)
print(X_selected.shape)


(100, 2)


In [10]:
###--- Data pre-processing and cleaning for training model---###

def prepare_df1(df):
    
    #Calculating the duration
    df = df.reset_index()
    df = df.dropna(subset=['PlanFinishDate'])
    mi_holidays = holidays.UnitedStates(state='MI')
    start = df['PlanStartDate'].astype(str)
    end = df['ActualFinishDate'].astype(str)
    bus = []
    holi = []
    dur = []
    start_list = start.tolist()
    end_list = end.tolist()
    for i in range(len(start_list)):
        holi.append(len(mi_holidays[start_list[i]: end_list[i]]))
        bus.append(np.busday_count(start_list[i], end_list[i]))

    zip_object = zip(bus, holi)
    for list1_i, list2_i in zip_object:
        dur.append(list1_i-list2_i)
    df['dur'] = dur
    variables_to_remove = ['index','SharePointID', 'ID', 'ProjectName', 'CreateDate', 'PlanStartDate',
             'PlanFinishDate', 'StatusDate', 'ProjectDescription', 'Status', 'StatusDate', 'ProjectDescription',
             'Status', 'RefrenceNumber', 'ProjectStage', 'CancelationReason', 'ActualFinishDate', 'Unit',
             'SolicitationStatus', 'LastUpdatedBy', 'LastUpdateDate', 'LastUpdatedField', 'SolicitationNum',
             'SensitivityIndicator', 'SourcingCategory', 'DollarAmount']

    df = df.drop(variables_to_remove, axis = 1)
    df = df.rename(columns={"dur": "Duration"})

    # handling NaN values
    imputer = CategoricalImputer()
    data = np.array(df['Planned'], dtype=object)
    data2 = np.array(df['CreatedBy'], dtype=object)
    data3 = np.array(df['BuyerName'], dtype=object)

    df['Planned'] = imputer.fit_transform(data)
    df['CreatedBy'] = imputer.fit_transform(data2)
    df['BuyerName'] = imputer.fit_transform(data3)
    df = df.dropna()
    df = df.reset_index()
    df = df.drop('index', axis = 1)
    return df

def met_sla(df):
    
    if df['Duration'] <= df['SLA']:
        val = 1
    elif df['Duration'] > df['SLA']:
        val = 0
    return val

model1_df = prepare_df1(df)
model1_df['Met_SLA'] = model1_df.apply(met_sla, axis=1)
model1_df.head()


,ProjectColor,ProjectType,CreatedBy,WorkPerformedFor,BuyerName,Hours,Division,SLA,Planned,CategoryID,FedReview,ITComponent,LevelCalc,Duration,Met_SLA
0,Green,CN,keilenn,071,JY1,0.0,PROF,63,Yes,50,False,False,3,5,1
1,Green,RFP,Gibrilm,000,YMD,120.0,COMM,138,Yes,185,False,False,2,54,1
2,Green,RFP,Gibrilm,000,YMD,120.0,COMM,138,No,184,False,False,3,58,1
3,Green,RFP,Gibrilm,591,YMD,120.0,COMM,138,No,301,False,False,2,101,1
4,Green,RFP,Gibrilm,431,SR4,120.0,COMM,138,No,143,False,False,4,172,0


In [11]:
###--- Functions that prepares data for model--> turns data entries to a form the model can read ---###

def prepare_inputs(X_train, X_test):
    oe = OrdinalEncoder(handle_unknown = "use_encoded_value", unknown_value= 569)
    oe.fit(X_train)
    X_train_enc = oe.transform(X_train)
    X_test_enc = oe.transform(X_test)
    return X_train_enc, X_test_enc

# prepare target
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_test_enc

# feature selection
def select_features(X_train, y_train, X_test):
    fs = SelectKBest(score_func=chi2, k='all')
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

X = model1_df.iloc[:, :-1]
y = model1_df.iloc[:,-1]
    # format all fields as string
X = X.astype(str)
np.set_printoptions(threshold=sys.maxsize)
X


,ProjectColor,ProjectType,CreatedBy,WorkPerformedFor,BuyerName,Hours,Division,SLA,Planned,CategoryID,FedReview,ITComponent,LevelCalc,Duration
0,Green,CN,keilenn,071,JY1,0.0,PROF,63,Yes,50,False,False,3,5
1,Green,RFP,Gibrilm,000,YMD,120.0,COMM,138,Yes,185,False,False,2,54
2,Green,RFP,Gibrilm,000,YMD,120.0,COMM,138,No,184,False,False,3,58
3,Green,RFP,Gibrilm,591,YMD,120.0,COMM,138,No,301,False,False,2,101
4,Green,RFP,Gibrilm,431,SR4,120.0,COMM,138,No,143,False,False,4,172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3454,Green,CN,lugibihlc,231,VAH,4.0,COMM,50,No,141,False,False,2,3
3455,Green,CN,lugibihlc,231,VAH,4.0,PROF,50,No,75,False,False,2,7
3456,Green,CN,lugibihlc,271,SW2,4.0,PROF,50,No,273,False,False,4,4
3457,Green,CN,lugibihlc,591,SW2,1.0,PROF,50,No,4,False,False,4,3


In [12]:
###--- Running algorithm that selects the best features ---###


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)
# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train_enc, y_train_enc, X_test_enc)
# what are scores for the features
for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, fs.scores_[i]))

unique_elements, counts_elements = np.unique(y_train_enc, return_counts=True)


Feature 0: 34.028406
Feature 1: 174.764623
Feature 2: 104.597134
Feature 3: 0.052861
Feature 4: 246.592093
Feature 5: 969.688138
Feature 6: 4.141410
Feature 7: 29.975731
Feature 8: 54.726723
Feature 9: 4.596208
Feature 10: 3.495505
Feature 11: 5.668895
Feature 12: 7.639144
Feature 13: 2391.797025


In [13]:
X_train_enc = np.delete(X_train_enc, np.s_[2, 6, 10, 11, 13], 1)
X_test_enc = np.delete(X_test_enc, np.s_[2, 6, 10, 11, 13], 1)

In [14]:
def evaluate_model(X, y, model):
    # define evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    # evaluate model
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores

#run algorithm multiple times to get desired accuracy
desired_accu = 0
while desired_accu < 0.832:

    
    #oversample training data to avoid skewness
    oversample = SMOTE(random_state=42, sampling_strategy={0:2500, 1:2500})
    X_s, y_s = oversample.fit_resample(X_train_enc, y_train_enc)
    #X_test = X_test.to_numpy()

    model = RandomForestClassifier(n_estimators=1000).fit(X_s, y_s)

    model = model.fit(X_s, y_s)
    
    # evaluate the model
    scores = evaluate_model(X_test_enc, y_test_enc, model)
    desired_accu = np.mean(scores)
    print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

C:\Users\gibrilm\AppData\Local\Continuum\anaconda3\lib\site-packages\imblearn\utils\_validation.py:300: UserWarning: After over-sampling, the number of samples (2500) in class 0 will be larger than the number of samples in the majority class (class #1 -> 1924)
  f"After over-sampling, the number of samples ({n_samples})"
C:\Users\gibrilm\AppData\Local\Continuum\anaconda3\lib\site-packages\imblearn\utils\_validation.py:300: UserWarning: After over-sampling, the number of samples (2500) in class 1 will be larger than the number of samples in the majority class (class #1 -> 1924)
  f"After over-sampling, the number of samples ({n_samples})"


Mean Accuracy: 0.851 (0.017)


In [15]:
###--- Save the trained model ---###

filename = 'model1try1.sav'
pickle.dump(model, open(filename, 'wb'))